# This portion is concerned with data preparation. 

# Please go to https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification  to download the raw data from kaggle.

# And place the downloaded folder in the main directory with the name 'Data'


# Import all libraries

In [1]:
import tensorflow as tf
import numpy as np
import scipy
import glob
from PIL import Image
import os
import matplotlib.pyplot as plt
from keras import layers
from keras.layers import (Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, 
                          Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D)
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from IPython.display import SVG

from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from pydub import AudioSegment
import shutil
from keras.preprocessing.image import ImageDataGenerator
import random
import librosa

Using TensorFlow backend.
/opt/anaconda3/lib/python3.7/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


# Creating directories to store the project data
# We are creating a directory for each folder. You could change the directory if you want 

In [7]:
os.makedirs('train')
os.makedirs('test')

genres = 'blues classical country disco hiphop metal pop reggae rock'
genres = genres.split()
for g in genres:
    path_audio = os.path.join('audio3sec',f'{g}')
    os.makedirs(path_audio)
    path_train = os.path.join('train',f'{g}')
    path_test = os.path.join('test',f'{g}')
    os. makedirs(path_train)
    os. makedirs(path_test)

# Segment audio each audio file into 10 files of 3 seconds duration

In [11]:
from pydub import AudioSegment
i = 0
for g in genres:
    j=0
    print(f"{g}")
    
    for filename in os.listdir(os.path.join('Data/genres_original',f"{g}")):

        song  =  os.path.join(f'Data/genres_original/{g}',f'{filename}')
        j = j+1
    
        for w in range(0,10):
            i = i+1

            t1 = 3*(w)*1000
            t2 = 3*(w+1)*1000
            newAudio = AudioSegment.from_wav(song)
            new = newAudio[t1:t2]
            new.export(f'audio3sec/{g}/{g+str(j)+str(w)}.wav', format="wav")

blues
classical
country
disco
hiphop
metal
pop
reggae
rock


# Transform the audio files into images

In [5]:
for g in genres:
    j = 0
    print(g)
    for filename in os.listdir(os.path.join('audio3sec',f"{g}")):
        song  =  os.path.join(f'audio3sec/{g}',f'{filename}')
        j = j+1
    
        y,sr = librosa.load(song,duration=3)
    
        mels = librosa.feature.melspectrogram(y=y,sr=sr)
        fig = plt.Figure()
        canvas = FigureCanvas(fig)
        p = plt.imshow(librosa.power_to_db(mels,ref=np.max))
        plt.savefig(f'train/{g}/{g+str(j)}.png')

# Split data into train and test dataset

In [6]:
directory = "train/"
for g in genres:
    filenames = os.listdir(os.path.join(directory,f"{g}"))
    random.shuffle(filenames)
    test_files = filenames[0:100]

    for f in test_files:

        shutil.move(directory + f"{g}"+ "/" + f,"test/" + f"{g}")

# The already pre-processed dataset could be found on on the below link
# https://drive.google.com/drive/folders/1AQU8whF8VYNGBVG_3SWsn3sgqbk7j-4o